Necessary Imports

In [1]:
import os
import sys
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader

sys.path.append(os.path.abspath(os.path.join('..', '..', 'models', 'cnn')))
from cnn import CNN
from multilabel_cnn import MultiLabelCNN


Implement a function load mnist data() that extracts images from the
dataset folders and organizes them into separate lists for images and labels
corresponding to the train, validation, and test splits. Ensure that the
images are loaded from their respective folders without any overlap or
mixing.

In [2]:
def one_hot_encode_digit(digit):
    """Creates an 11-dimensional one-hot vector for a single digit or blank."""
    one_hot = np.zeros(11)
    if digit == '*':
        one_hot[10] = 1 
    else:
        one_hot[int(digit)] = 1
    return one_hot

def one_hot_encode_label(label):
    """Encodes the label (up to three digits) as a 33-dimensional vector."""
    label_str = str(label).ljust(3, '*')
    one_hot_encoded = np.concatenate([one_hot_encode_digit(digit) for digit in label_str])
    return one_hot_encoded


def load_mnist_data(base_path='../../data/external/double_mnist', use_label_count=False):
    images = {
        'train': [],
        'val': [],
        'test': []
    }
    labels = {
        'train': [],
        'val': [],
        'test': []
    }

    splits = ['train', 'val', 'test']
    for split in splits:
        split_path = os.path.join(base_path, split)
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if os.path.isdir(label_path):
                for img_name in os.listdir(label_path):
                    img_path = os.path.join(label_path, img_name)
                    img = Image.open(img_path).convert('L')
                    img_array = np.array(img)
                    images[split].append(img_array)

                    if use_label_count:
                        if label == "0":
                            label_count = 0
                        else:
                            label_count = len(label)
                        labels[split].append(label_count)
                    else:
                        labels[split].append(one_hot_encode_label(label))

    return images, labels

images, labels = load_mnist_data(use_label_count=True)


Create a class called MultiMNISTDataset that will be used to create
dataloaders for training and evaluation purposes.

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MultiMNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        img = torch.tensor(img, dtype=torch.float32) / 255.0  

        if self.transform:
            img = self.transform(img)

        return img, label

transform = transforms.Compose([
    transforms.ToPILImage(),         
    transforms.Resize((28, 28)),     
    transforms.ToTensor(),         
])

train_dataset = MultiMNISTDataset(images['train'], labels['train'], transform=transform)
val_dataset = MultiMNISTDataset(images['val'], labels['val'], transform=transform)
test_dataset = MultiMNISTDataset(images['test'], labels['test'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Implement the CNN Class

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(task='classification', num_classes=4).to(device)
model.fit(train_loader, val_loader, epochs=5, lr=0.001)

Training Epoch 1/5:  24%|██▍       | 95/394 [00:03<00:11, 26.05it/s]


KeyboardInterrupt: 

In [ ]:

from sklearn.metrics import accuracy_score
def calculate_accuracy(model, test_loader):
    predictions = model.predict(test_loader)
    model.eval()
    true_labels = []
    for _, labels in test_loader:
        true_labels.append(labels)

    true_labels = torch.cat(true_labels).cpu()
    print(true_labels)
    print(torch.round(predictions).cpu())

    accuracy = accuracy_score(true_labels, torch.round(predictions).cpu())
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy
accuracy = calculate_accuracy(model, test_loader)

tensor([3, 3, 3,  ..., 3, 3, 3])
tensor([3, 3, 3,  ..., 3, 3, 3])
Test Accuracy: 1.0000


In [5]:
images, labels = load_mnist_data(use_label_count=False)

train_dataset = MultiMNISTDataset(images['train'], labels['train'], transform=transform)
val_dataset = MultiMNISTDataset(images['val'], labels['val'], transform=transform)
test_dataset = MultiMNISTDataset(images['test'], labels['test'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiLabelCNN(device=device).to(device)
model.fit(train_loader, val_loader, epochs=40, lr=0.001)

Training Epoch 1/40: 100%|██████████| 394/394 [00:11<00:00, 35.55it/s]


Epoch [1/40], Training Loss: 5.0922


In [ ]:
def exact_match_accuracy(preds, labels):
    """Calculates the exact match accuracy between predictions and labels."""
    return (preds == labels).all(dim=1).float().mean().item()

def hamming_accuracy(preds, labels):
    """Calculates the Hamming accuracy for multi-label predictions."""
    return (preds == labels).float().mean().item()


def evaluate_model(model, test_loader):
    preds = model.predict(test_loader)
    labels = torch.cat([labels for _, labels in test_loader])
    print(preds[12], labels[12])

    exact_acc = exact_match_accuracy(preds, labels)
    hamming_acc = hamming_accuracy(preds, labels)
    print(f"Exact Match Accuracy: {exact_acc * 100:.2f}%")
    print(f"Hamming Accuracy: {hamming_acc * 100:.2f}%")

evaluate_model(model, test_loader)


tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]) tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
Exact Match Accuracy: 28.98%
Hamming Accuracy: 95.26%
